<a href="https://colab.research.google.com/github/dominiksakic/zero_to_hero/blob/main/basics_06_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- source: https://www.youtube.com/watch?v=kCc8FmEb1nY&t=126s

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [4]:

# get data
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-07-16 23:04:49--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-07-16 23:04:49 (16.8 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [5]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [ ]:
# make decoder, encoder
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch:i  for i, ch in enumerate(chars)}
itos = {i : ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
print(encode(text[:50]))
print(decode(encode(text[:50])))
print(f"Vocav size: {vocab_size}")

In [ ]:
# Tokenize data, and create test/val
data = torch.tensor(encode(text), dtype=torch.long)

n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
# excursion into how the model predicts next token from one sentence
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]  #<---
    target = y[t]
    print(f'timestep {t}: when input is {context} the target is {target}')

timestep 0: when input is tensor([18]) the target is 47
timestep 1: when input is tensor([18, 47]) the target is 56
timestep 2: when input is tensor([18, 47, 56]) the target is 57
timestep 3: when input is tensor([18, 47, 56, 57]) the target is 58
timestep 4: when input is tensor([18, 47, 56, 57, 58]) the target is 1
timestep 5: when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
timestep 6: when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
timestep 7: when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


  - Result is that the model learns to complete from various lengths.
  - from one characters up to 8.


In [ ]:
# lets make the example more complex by introducing a batch dimension
torch.manual_seed(1337)
batch_size = 4

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+ block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]

    print(f"Batch {b}: when context is {context}, target is {target}")
  print("\n")

Batch 0: when context is tensor([24]), target is 43
Batch 0: when context is tensor([24, 43]), target is 58
Batch 0: when context is tensor([24, 43, 58]), target is 5
Batch 0: when context is tensor([24, 43, 58,  5]), target is 57
Batch 0: when context is tensor([24, 43, 58,  5, 57]), target is 1
Batch 0: when context is tensor([24, 43, 58,  5, 57,  1]), target is 46
Batch 0: when context is tensor([24, 43, 58,  5, 57,  1, 46]), target is 43
Batch 0: when context is tensor([24, 43, 58,  5, 57,  1, 46, 43]), target is 39


Batch 1: when context is tensor([44]), target is 53
Batch 1: when context is tensor([44, 53]), target is 56
Batch 1: when context is tensor([44, 53, 56]), target is 1
Batch 1: when context is tensor([44, 53, 56,  1]), target is 58
Batch 1: when context is tensor([44, 53, 56,  1, 58]), target is 46
Batch 1: when context is tensor([44, 53, 56,  1, 58, 46]), target is 39
Batch 1: when context is tensor([44, 53, 56,  1, 58, 46, 39]), target is 58
Batch 1: when context is 

In [ ]:
# create a baseline model/bigram
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    logits = self.token_embedding_table(idx) # (B,T,C)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape

      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
      # idx is (B, T) array of indices in the current context
      for _ in range(max_new_tokens):
          # get the predictions
          logits, loss = self(idx)
          # focus only on the last time step
          logits = logits[:, -1, :] # becomes (B, C)
          # apply softmax to get probabilities
          probs = F.softmax(logits, dim=-1) # (B, C)
          # sample from the distribution
          idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
          # append sampled index to the running sequence
          idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
      return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss.item())

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=50)[0].tolist()))

torch.Size([256, 65])
4.648044586181641

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLER


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
# train the baseline
batch_size = 32

for s in range(100000):
  # trainig data
  xb, yb = get_batch('train')

  # Forward pass
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)

  #Backward pass
  loss.backward()
  optimizer.step()

print(loss.item())

2.4081521034240723


In [ ]:
# sample after training
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))



MAUEdingn I
GBENoutl, Thasuriopro sorllll le.

Mat,
FI s ICKI ambe mithevis LLEShe ysste ar s blllyorswon
Clmbe t uruk
CLLarf w p ar lye twemen ulif hercefowive:
YBOUSTIOVO: gh ced s p ay anore iveatothe ierave yanccu wind s; oalllak omad ste?
h;

JUThow h llde iouge thes whe yomeathistlieis moma hit me o ind.

F hik, thite:
TRThe hal at w!
Whase t ma T:

Bareomast yethin athe stt geloupr msh f wh n
Yorinkeshave pan t,
NGAnthe or,
Wh tro joullieallisube:
Fin matthese V:
Aporn geng y ll yr mofor


In [ ]:
xv, yv = get_batch("val")
_, loss = m(xv, yv)

print(f"Validation loss: {loss}")

Validation loss: 2.419926404953003


# Transformers


In [ ]:
# Gensis of Transformer (weighted aggregation)
import torch

torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
print('Starting Tensor a')
print({a})

a = a / torch.sum(a, 1, keepdim=True)
print(f'\nShape of a: {a.shape}')
print('Divide values by the sum along the 1 axis: ')
print({a})

b = torch.randint(0,10,(3,2)).float()
print(f'\nShape of b: {b.shape}')
print('Starting tensor b is')
print(b)

c = a @ b
print('\nresult of c is')
print(c)


Starting Tensor a
{tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])}

Shape of a: torch.Size([3, 3])
Divide values by the sum along the 1 axis: 
{tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])}

Shape of b: torch.Size([3, 2])
Starting tensor b is
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])

result of c is
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
print(f'Batch 0')
print(x[0])

Batch 0
tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])


In [ ]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    print(f'Batch: {b}')
    for t in range(T):
      xprev = x[b, :t+1]
      xbow[b,t] = torch.mean(xprev, 0)
      print(f'time step {t}, after aggregation: {xbow[b,t]}')

    if b == 0:
      break

Batch: 0
time step 0, after aggregation: tensor([ 0.1808, -0.0700])
time step 1, after aggregation: tensor([-0.0894, -0.4926])
time step 2, after aggregation: tensor([ 0.1490, -0.3199])
time step 3, after aggregation: tensor([ 0.3504, -0.2238])
time step 4, after aggregation: tensor([0.3525, 0.0545])
time step 5, after aggregation: tensor([ 0.0688, -0.0396])
time step 6, after aggregation: tensor([ 0.0927, -0.0682])
time step 7, after aggregation: tensor([-0.0341,  0.1332])


- For each time step we want to have the previous timesteps accumulated(mean)

In [ ]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
print(f'shape of wei: {wei.shape}')
print(f'wei is: {wei}')

wei = wei / wei.sum(1, keepdim=True)
print(f'\nwei after normalization: {wei}')

xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
print(f'\nxbow2 batch 0 after multiplication: {xbow2[0]}')


shape of wei: torch.Size([8, 8])
wei is: tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

wei after normalization: tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

xb

In [ ]:
import torch.nn.functional as F
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
print(f'tril: {tril}')

wei = torch.zeros((T,T))
print(f'\nwei: {wei}')

wei = wei.masked_fill(tril == 0, float('-inf'))
print(f'\nwei after masking: {wei}')

wei = F.softmax(wei, dim=-1)
print(f'\nwei after softmax: {wei}')

xbow3 = wei @ x
print(f'\nxbow3 batch 0: {xbow3[0]}')

tril: tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

wei: tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

wei after masking: tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -

In [16]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

# single Head
head_size = 16
key = nn.Linear(C, head_size, bias=False) # what I am asking about
query = nn.Linear(C, head_size, bias=False) # waht others offer
value = nn.Linear(C, head_size, bias=False) # what others contain
# The original input vector 32-dimension gets mapped inot a smaller space
k = key(x)   # B, T, 16
q = query(x) # B, T, 16

# Attention scores for every position with each other
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

# mask future tokens, for autoregressive models
tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

# weighted sums of values/contextualization
v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [12]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [31]:
#Solving the transpose mystery(for me)
B,T,C = 1,2,4
x = torch.randn(B,T,C)
print(f'x: \n{x}\n')

head_size = 3
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)
q = query(x)

print(f'key: \n{key}\n')
print(f'k: \n{k}\n')

# Question why cant I just do q @ k  = (1,2,3) @ (1,2,3)
# Answer the inner dimension dont align (2,3) @ (2,3)
"""
              x11 x12 x123
              x21 x22 x223
x11 x12 x123
x21 x22 x223

== not possible thus swap the last layer (-1) and the one before(-2)

              x11 x21
              x12 x22
              x13 x23
x12 x22 x123
x21 x22 x223
"""
print(f'before: k values: \n{k}\n')
print(f'after: k values: \n{k.transpose(-2, -1)}\n')


x: 
tensor([[[ 1.0729, -0.5504,  0.4417,  1.8560],
         [-1.6223, -1.8155, -0.8447, -2.1575]]])

key: 
Linear(in_features=4, out_features=3, bias=False)

k: 
tensor([[[-0.8246,  0.3190,  0.2538],
         [-0.1150, -0.0447,  0.1061]]], grad_fn=<UnsafeViewBackward0>)

before: k values: 
tensor([[[-0.8246,  0.3190,  0.2538],
         [-0.1150, -0.0447,  0.1061]]], grad_fn=<UnsafeViewBackward0>)

after: k values: 
tensor([[[-0.8246, -0.1150],
         [ 0.3190, -0.0447],
         [ 0.2538,  0.1061]]], grad_fn=<TransposeBackward0>)



Notes from Andrej:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [32]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [33]:
k.var()

tensor(1.3187)

In [34]:
q.var()

tensor(0.6392)

In [35]:
wei.var()

tensor(0.3140)

In [36]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [37]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])